# Belajar Docker

## Image [build]
```
docker build --help
Usage:	docker build [OPTIONS] PATH | URL |
```
Options yang sering dipakai:
- `--build-arg list` Set build-time variables
- `--no-cache` Do not use cache when building the image
- `-t, --tag list` Name and optionally a tag in the 'name:tag' format
- `-f, --file string` Name of the Dockerfile (Default is 'PATH/Dockerfile')


## Container [run]

```
docker run --help
Usage:	docker run [OPTIONS] IMAGE [COMMAND] [ARG...]
```
Options yang sering dipakai:
- `-d, --detach` Run container in background and print container ID
- `-e, --env list` Set environment variables
- `--env-file list` Read in a file of environment variables
- `-i, --interactive` Keep STDIN open even if not attached
- `--network network` Connect a container to a network
- `-p, --publish list` Publish a container's port(s) to the host
- `--rm` Automatically remove the container when it exits
- `-v, --volume list` [Bind mount a volume](https://docs.docker.com/storage/)
- `--name string` Assign a name to the container


## Container [operation]
```
docker container --help
Usage:	docker container COMMAND
```
Options yang sering dipakai:
- `ls, ls -a` List containers atau `docker ps, docker ps -a`
- `exec` Run a command in a running container
- `inspect` Display detailed information on one or more containers
- `logs` Fetch the logs of a container
- `restart` Restart one or more containers
- `rm` Remove one or more containers
- `start` Start one or more stopped containers
- `stop` Stop one or more running containers


## Push Docker Image ke Registry
```
>> gcloud auth configure-docker
>> docker tag hello asia.gcr.io/tiket-0818/test/hello
>> docker push asia.gcr.io/tiket-0818/test/hello
```


## Pull Docker Image dari Registry
```
>> docker pull alpine
>> docker asia.gcr.io/tiket-0818/test/hello
```


## Bersih bersih
Remove container
```
>> docker rm CONTAINER
```
Remove images
```
>> docker rmi REPOSITORY:TAG
```
Remove all
```
docker system prune -a --volumes
```

# Contoh 1 - Menggunakan docker

## Pull
[Alpine Linux](https://hub.docker.com/_/alpine)

```
>> docker pull alpine
```

## Cek Images
```
>> docker images
```

## Cek Container
```
>> docker container ls
```

## Run Container
```
>> docker run alpine:latest

>> docker run --name my_alpine alpine:latest
>> docker run -i --rm alpine:latest
```

# Contoh 2 - Membuat Dockerfile
- [Documents Dockerfile](https://docs.docker.com/engine/reference/builder/)

## Membuat Image dengan Dockerfile
```
from flask import Flask, request, jsonify
import os

app = Flask(__name__)


@app.route('/', methods=['GET'])
def data():
    data = {
        "Hello": os.environ.get("NAME", "World")
    }
    return jsonify(data)


if __name__ == '__main__':
    app.run(host="0.0.0.0", port=7000, debug=True)

```

### Requirements
```
Flask==1.1.2
```

### Dockerfile
```
FROM python:3.6-alpine
RUN mkdir -p /usr/src/app
COPY . /usr/src/app
WORKDIR /usr/src/app
RUN pip install -r requirements.txt
CMD ["python", "app.py"]
```

### Build Image
```
>> docker build -t hello:latest .
```

### Run Image (container)
```
>> docker run -d -p 8000:7000 --name hello_world hello
```

# Contoh 3 - Env, Port, Volume, Network


### Requirements
```
google-cloud-storage==1.28.1
Flask==1.1.2
mongoengine==0.20.0
scikit-learn==0.23.1
```

### Environment Variable
```
GOOGLE_APPLICATION_CREDENTIALS=/usr/src/app/my-sa.json
BUCKET_NAME=sandbox_datascientist
BUCKET_DEST=temp/rf_model.pkl
PROJECT_ID=tiket-0818
MODEL_NAME=rf_model.pkl
MONGO_HOST=localhost
MONGO_PORT=27017
MONGO_DB=hotel_
```

### Dockerfile
```
FROM python:3.6.10-slim-buster
RUN mkdir -p /usr/src/app
COPY . /usr/src/app
WORKDIR /usr/src/app
RUN pip install -r requirements.txt
CMD ["python", "app.py"]
```

### Build Image
```
>> docker build -t hotel_cancel:latest --no-cache .
```

### Run Image
```
>> docker run --name hotel_app --env-file docker.env -p 7000:7000 -d -v /Users/ulwan/Documents/project/my-sa.json:/usr/src/app/my-sa.json hotel_cancel
```

### Network
```
docker network create app_network
docker network connect app_network mongo
docker network connect app_network hotel_app
```

### Volume
```
docker volume create mongo_db

docker run -d --name mongo -v mongo_db:/data/db mongo
```

# Docker Compose

```
version: "3.7"

services:
  mongo:
    container_name: mongo
    image: mongo:latest
    ports:
      - 27017:27017
    networks:
      - app_network
    volumes:
      - mongo_db:/data/db
  hotel_app:
    container_name: hotel_app
    image: hotel_cancel:latest
    ports:
      - 8000:7000
    depends_on:
      - mongo
    environment:
      - GOOGLE_APPLICATION_CREDENTIALS=/usr/src/app/my-sa.json
      - BUCKET_NAME=sandbox_datascientist
      - BUCKET_DEST=temp/rf_model.pkl
      - PROJECT_ID=tiket-0818
      - MODEL_NAME=rf_model.pkl
      - MONGO_HOST=mongo
      - MONGO_PORT=27017
      - MONGO_DB=hotel_
    networks:
      - app_network
    volumes:
      - /Users/ulwan/Documents/project/my-sa.json:/usr/src/app/my-sa.json
networks:
  app_network:
    name: app_network

volumes:
  mongo_db:
    name: mongo_db

```